In [136]:
#https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/performance/post_training_integer_quant.ipynb
import tensorflow as tf
print(tf.__version__)

DIMX = 4096
DIMY = 8192

class Model(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec(shape=[DIMX, DIMY], dtype=tf.float32, name=None), 
                                  tf.TensorSpec(shape=[DIMY, 1], dtype=tf.float32, name=None)])
    def func1(self, x, y):
        return tf.matmul(x,y)

model = Model()

converter = tf.lite.TFLiteConverter.from_concrete_functions([model.func1.get_concrete_function()], model)
tflite_model = converter.convert() 

with open('genGEMV_float32.tflite', 'wb') as f:
    f.write(tflite_model)

interpreter = tf.lite.Interpreter(model_content=tflite_model)
signatures = interpreter.get_signature_list()
print(signatures)

2.13.0
INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmp5wy5l5kl\assets


INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmp5wy5l5kl\assets


{'serving_default': {'inputs': ['x', 'y'], 'outputs': ['output_0']}}


In [137]:
#Dynamic Range Quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_dynamic = converter.convert()

with open('genGEMV_dynamic.tflite', 'wb') as f:
  f.write(tflite_model_dynamic)

INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmp65jbt8uw\assets


INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmp65jbt8uw\assets


In [138]:
#Float16 Quantization
converter.target_spec.supported_types = [tf.float16]
tflite_model_float16 = converter.convert()

with open('genGEMV_float16.tflite', 'wb') as f:
  f.write(tflite_model_float16)

INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmp6dr5rypv\assets


INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmp6dr5rypv\assets


In [141]:
#Integer Float Fallback Quantization
converter = tf.lite.TFLiteConverter.from_concrete_functions([model.func1.get_concrete_function()], model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset():
    for i in range(10): # create fake data
        fake_data_x = tf.random.normal([DIMX, DIMY])
        fake_data_y = tf.random.normal([DIMY, 1])
        #yield [fake_data_x, fake_data_y] #reverse order of inputs here!
        yield [fake_data_y, fake_data_x]
converter.representative_dataset = representative_dataset

tflite_model_integer_fallback = converter.convert()

with open('genGEMV_integer_fallback.tflite', 'wb') as f:
  f.write(tflite_model_integer_fallback)

INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmpnwrqqqv7\assets


INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmpnwrqqqv7\assets


In [142]:
#integer-only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_integer_only = converter.convert()

with open('genGEMV_integer_only.tflite', 'wb') as f:
  f.write(tflite_model_integer_only)

INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmpix_8bs_d\assets


INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmpix_8bs_d\assets
